In [1]:
from pyspark.sql import SparkSession, functions as f
import pyspark.sql.types as t

In [2]:
spark = SparkSession.\
    builder.\
    appName("freaking_driver").\
    config("spark.jars", "postgresql-42.4.0.jar").\
    getOrCreate()

22/07/19 01:23:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
connector = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/vladbaran")\
    .option("user", "postgres")\
    .option("password", "klondike3711")\
    .option("driver", "org.postgresql.Driver")

In [4]:
actor = connector.option("dbtable", "actor").load()

In [5]:
def table_reader(table_name:str):
    return connector.option("dbtable", table_name).load()

In [6]:
actor = table_reader("actor")
address = table_reader("address")
category = table_reader("category")
city = table_reader("city")
country = table_reader("country")
customer = table_reader("customer")
film = table_reader("film")
film_actor = table_reader("film_actor")
film_category = table_reader("film_category")
inventory = table_reader("inventory")
language = table_reader("language")
payment = table_reader("payment")
rental = table_reader("rental")
staff = table_reader("staff")
store = table_reader("store")

In [86]:
rental.printSchema()

root
 |-- rental_id: integer (nullable = true)
 |-- rental_date: timestamp (nullable = true)
 |-- inventory_id: integer (nullable = true)
 |-- customer_id: short (nullable = true)
 |-- return_date: timestamp (nullable = true)
 |-- staff_id: short (nullable = true)
 |-- last_update: timestamp (nullable = true)



In [7]:
#1
category.join(film_category, on="category_id")\
    .groupBy("name")\
    .count()\
    .orderBy(-f.col("count").cast(t.FloatType()))\
    .show()

+-----------+-----+
|       name|count|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



In [8]:
#2
actor.join(film_actor, on="actor_id")\
    .join(film, on="film_id")\
    .withColumn("full_name", f.concat_ws(' ', "first_name", "last_name"))\
    .groupBy("full_name")\
    .agg(
    f.sum("rental_duration").alias("total")
    )\
    .orderBy(-f.col("total"))\
    .show(10)

+--------------+-----+
|     full_name|total|
+--------------+-----+
|   SUSAN DAVIS|  242|
|GINA DEGENERES|  209|
|   WALTER TORN|  201|
|   MARY KEITEL|  192|
|MATTHEW CARREY|  190|
| GROUCHO DUNST|  183|
| ANGELA HUDSON|  183|
| SANDRA KILMER|  181|
|   HENRY BERRY|  180|
|      UMA WOOD|  179|
+--------------+-----+
only showing top 10 rows



In [9]:
#3
category.join(film_category, on="category_id")\
    .join(film, on="film_id")\
    .join(inventory, on="film_id")\
    .join(rental, on="inventory_id")\
    .join(payment, on="rental_id")\
    .groupBy("name")\
    .agg(
    f.sum("amount").alias("sum")
    )\
    .orderBy(-f.col("sum")).limit(1).show()

+------+-------+
|  name|    sum|
+------+-------+
|Sports|5314.21|
+------+-------+



In [35]:
#4
film.join(inventory, on="film_id", how="left").filter("inventory_id is null").select("title").orderBy("title").show(42)

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
|           GUMP DATE|
|       HATE HANDICAP|
|         HOCUS FRIDA|
|    KENTUCKIAN GIANT|
|    KILL BROTHERHOOD|
|         MUPPET MILE|
|      ORDER BETRAYED|
|       PEARL DESTINY|
|     PERDITION FARGO|
|       PSYCHO SHRUNK|
|   RAIDERS ANTITRUST|
|       RAINBOW SHOCK|
|       ROOF CHAMPION|
|       SISTER FREDDY|
|         SKY MIRACLE|
|    SUICIDES SILENCE|
|        TADPOLE PARK|
|    TREASURE COMMAND|
|   VILLAIN DESPERATE|
|        VOLUME HOUSE|
|          

In [63]:
#5
tbl = category.join(film_category, on="category_id")\
    .join(film, on="film_id")\
    .join(film_actor, on="film_id")\
    .join(actor, on="actor_id")\
    .where(f.col("name") == "Children")\
    .withColumn("full_name", f.concat_ws(' ', "first_name", "last_name"))\
    .groupBy("full_name").count()

In [64]:
filter_topcount = tbl.select(f.col("count")).\
    orderBy(-f.col("count").cast(t.FloatType()))\
    .distinct()\
    .limit(3)\
    .rdd.map(lambda row : row[0]).collect()

In [65]:
tbl.filter(f.col("count").isin(filter_topcount)).show()

+-------------+-----+
|    full_name|count|
+-------------+-----+
|   MARY TANDY|    5|
|   RALPH CRUZ|    5|
|  WHOOPI HURT|    5|
|KEVIN GARLAND|    5|
| HELEN VOIGHT|    7|
|  SUSAN DAVIS|    6|
+-------------+-----+



In [71]:
#6 
tbl = customer.join(address, on="address_id")\
    .join(city, on="city_id")\
    .groupBy("city")\
    .agg(
    f.sum(f.col("active").cast(t.FloatType())).alias("inactive"),
    f.count(f.col("active").cast(t.FloatType())).alias("active")
)

In [80]:
tbl.withColumn("inactived", f.col("active")-f.col("inactive")).orderBy(-f.col("inactived")).show(20)

+------------------+--------+------+---------+
|              city|inactive|active|inactived|
+------------------+--------+------+---------+
|         Pingxiang|     0.0|     1|      1.0|
|       Szkesfehrvr|     0.0|     1|      1.0|
|  Charlotte Amalie|     0.0|     1|      1.0|
|         Najafabad|     0.0|     1|      1.0|
|           Wroclaw|     0.0|     1|      1.0|
|            Ktahya|     0.0|     1|      1.0|
|            Amroha|     0.0|     1|      1.0|
|   Southend-on-Sea|     0.0|     1|      1.0|
|           Bat Yam|     0.0|     1|      1.0|
|            Kamyin|     0.0|     1|      1.0|
|          Xiangfan|     0.0|     1|      1.0|
|            Daxian|     0.0|     1|      1.0|
|          Uluberia|     0.0|     1|      1.0|
|     Coatzacoalcos|     0.0|     1|      1.0|
|        Kumbakonam|     0.0|     1|      1.0|
|          Fengshan|     1.0|     1|      0.0|
|A Corua (La Corua)|     1.0|     1|      0.0|
|           El Alto|     1.0|     1|      0.0|
|          My

In [104]:
#7
tbl = city.join(address, on="city_id")\
    .join(customer, on="address_id")\
    .join(rental, on="customer_id")\
    .join(inventory, on="inventory_id")\
    .join(film, on="film_id")\
    .join(film_category, on="film_id")\
    .join(category, on="category_id")\
    .filter("return_date is not null")\
    .withColumnRenamed("city", "gorodok")\
    .select("name", "return_date", "rental_date", "gorodok")

In [114]:
result_1 = tbl.withColumn("timedelta", f.col("return_date") - f.col("rental_date"))\
    .filter(f.col("gorodok").like("A%") | f.col("gorodok").like("a%"))\
    .groupBy("name").agg(f.sum("timedelta").alias("period"))\
    .orderBy(-f.col("period")).limit(1)
    

In [115]:
result_2 = tbl.withColumn("timedelta", f.col("return_date") - f.col("rental_date"))\
    .filter(f.col("gorodok").like("%-%"))\
    .groupBy("name").agg(f.sum("timedelta").alias("period"))\
    .orderBy(-f.col("period")).limit(1)

In [116]:
result_1.union(result_2).show()

+-------+--------------------+
|   name|              period|
+-------+--------------------+
| Sports|INTERVAL '515 00:...|
|Foreign|INTERVAL '269 16:...|
+-------+--------------------+



In [120]:
#ну это было прям интересно! от души


AnalysisException: Table or view not found: actor; line 1 pos 21;
'Project ['actor_id]
+- 'UnresolvedRelation [actor], [], false
